In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preprocessing

## Importing the dataset

In [ ]:
df=pd.read_csv(os.path.join(dirname, filename))

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer

# Create imputers
imputer_categorical = SimpleImputer(strategy='most_frequent')
imputer_numeric = SimpleImputer(strategy='median')

df['Geography'] = imputer_categorical.fit_transform(df[['Geography']]).ravel()
df['HasCrCard'] = imputer_categorical.fit_transform(df[['HasCrCard']]).ravel()
df['IsActiveMember'] = imputer_categorical.fit_transform(df[['IsActiveMember']]).ravel()
df['Age'] = imputer_numeric.fit_transform(df[['Age']])

In [ ]:
df.isnull().sum()

In [ ]:
X=df.iloc[:,3:-1].values
y=df.iloc[:,-1].values

In [ ]:
print(X)

In [ ]:
print(y)

# Encoding categorical data

## Label Encoding the Gender Column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [ ]:
print(X)

## Label Encoding the Geography Column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building the ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ann= Sequential()
ann.add(Dense(units=6,activation='relu'))
ann.add(Dense(units=6,activation='relu'))
ann.add(Dense(units=1,activation='sigmoid'))

# Training the ANN

In [ ]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
ann.fit(X_train,y_train, batch_size=32,epochs=100)

# Part 4 - Making the predictions and evaluating the model

## Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)